In [131]:
import pandas as pd
import numpy as np

In [132]:
pd.set_option('display.max_columns', None)

In [133]:
df = pd.read_csv(r"C:\Users\mabin\Desktop\DataScienceClassNotes\Career247_Capstone_Project\Fraud_Detection_Project\data\raw\cleaned_data_v1.csv")

In [134]:
df.shape

(50000, 26)

In [135]:
df.head()

,transaction_id,customer_id,device_id,merchant_id,transaction_month,transaction_day,amount,payment_method,is_international,merchant_category,ip_address_risk_score,device_trust_score,txn_count_last_24h,avg_amount_last_24h,merchant_diversity_last_7d,device_change_flag,location_change_flag,authentication_method,otp_success_rate_customer,past_fraud_count_customer,past_disputes_customer,merchant_historical_fraud_rate,hour_of_day,day_of_week,is_weekend,is_fraud
0,1,1357,20937,720,1,18,18780.52,WALLET,0,Travel,0.243550,0.116606,7,6055.25,3,0,0,NONE,0.769328,1,2,0.069502,19,3,0,0
1,2,4095,29542,764,3,30,18955.06,CARD,0,Electronics,0.119761,0.030579,1,13366.74,5,0,1,OTP,0.468411,1,1,0.010010,2,5,1,0
2,3,3618,25236,671,1,4,10743.34,UPI,0,Fashion,0.565367,0.183770,0,7359.14,4,0,0,PIN,0.464356,0,1,0.118434,17,3,0,0
3,4,2755,29904,624,1,27,5054.75,CARD,0,Utilities,0.843842,0.717286,5,10923.57,6,0,0,PIN,0.635799,0,3,0.114947,16,5,1,0
4,5,2732,24151,501,2,20,14799.50,CARD,0,Gaming,0.202402,0.041616,11,6027.72,10,1,0,OTP,0.684301,2,0,0.014550,23,1,0,0


In [136]:
df.duplicated().sum()

np.int64(0)

In [137]:
df.nunique(dropna=False)

transaction_id                    50000
customer_id                        4000
device_id                          9946
merchant_id                         300
transaction_month                     3
transaction_day                      31
amount                            49412
payment_method                        4
is_international                      2
merchant_category                     6
ip_address_risk_score             50000
device_trust_score                50000
txn_count_last_24h                   12
avg_amount_last_24h               49167
merchant_diversity_last_7d           11
device_change_flag                    2
location_change_flag                  2
authentication_method                 4
otp_success_rate_customer         50000
past_fraud_count_customer             3
past_disputes_customer                5
merchant_historical_fraud_rate    49994
hour_of_day                          24
day_of_week                           7
is_weekend                            2


In [139]:
df.columns

Index(['transaction_id', 'customer_id', 'device_id', 'merchant_id',
       'transaction_month', 'transaction_day', 'amount', 'payment_method',
       'is_international', 'merchant_category', 'ip_address_risk_score',
       'device_trust_score', 'txn_count_last_24h', 'avg_amount_last_24h',
       'merchant_diversity_last_7d', 'device_change_flag',
       'location_change_flag', 'authentication_method',
       'otp_success_rate_customer', 'past_fraud_count_customer',
       'past_disputes_customer', 'merchant_historical_fraud_rate',
       'hour_of_day', 'day_of_week', 'is_weekend', 'is_fraud'],
      dtype='object')

## Feature Engineering

Feature engineering is a critical step in the fraud detection pipeline, aimed at transforming raw transactional data into meaningful variables that better capture hidden behavioural, temporal, and risk-based patterns associated with fraudulent activity. Since fraudulent transactions are extremely rare and often mimic legitimate behaviour, relying only on original dataset fields is insufficient for accurate prediction. Therefore, additional derived features were created to enhance the model’s ability to distinguish genuine transactions from suspicious ones.

In [ ]:
def feature_eng(data):
    
    # Transaction count per customers
    df['cust_txn_count'] = df.groupby('customer_id')['transaction_id'].transform('count')

    # Customer Fraud history count
    df['cust_fraud_count'] = df.groupby('customer_id')['is_fraud'].transform('sum')
    
    # Customer Fraud rate
    df['cust_fraud_rate'] = df['cust_fraud_count'] / (df['cust_txn_count'] + 1)
    
    # Calculating each customer average amount
    df['cust_avg_amt'] = df.groupby('customer_id')['amount'].transform('mean')
    
    # Calculating the amount deviation of customers
    df['amt_deviation'] = df['amount'] - df['cust_avg_amt']
    
    # Calculating the amount ratio of each customers
    df['amt_ratio'] = df['amount'] / (df['cust_avg_amt'] + 1)
    
    # New device count of each customers
    df['device_count'] = df.groupby('customer_id')['device_id'].transform('nunique')
    
    # Customers new device flag. It is if the customer have a new device
    df['new_device_flag'] = (df['device_count'] > 1).astype(int)
    
    # Merchant fraud rate of each merchant
    df['merchant_fraud_rate'] = df.groupby('merchant_id')['is_fraud'].transform('mean')
    
    # OTP risk rate of each customers
    df['otp_risk'] = 1 - df['otp_success_rate_customer']
    
    # combined risk for each customers
    df['combined_risk'] = (
    0.3 * df['merchant_fraud_rate'] +
    0.3 * df['cust_fraud_rate'] +
    0.2 * df['otp_risk'] +
    0.2 * df['is_international'])
    
    return data

In [141]:
df = feature_eng(df)

In [142]:
# Changing the is_fraud column position

col = df.pop('is_fraud')
df['is_fraud'] = col


In [143]:
df.sample(10)

,transaction_id,customer_id,device_id,merchant_id,transaction_month,transaction_day,amount,payment_method,is_international,merchant_category,ip_address_risk_score,device_trust_score,txn_count_last_24h,avg_amount_last_24h,merchant_diversity_last_7d,device_change_flag,location_change_flag,authentication_method,otp_success_rate_customer,past_fraud_count_customer,past_disputes_customer,merchant_historical_fraud_rate,hour_of_day,day_of_week,is_weekend,cust_txn_count,cust_fraud_count,cust_fraud_rate,cust_avg_amt,amt_deviation,amt_ratio,device_count,new_device_flag,merchant_fraud_rate,otp_risk,combined_risk,is_fraud
47123,47124,4276,25868,761,1,27,14224.59,CARD,0,Fashion,0.942511,0.831766,10,2757.67,4,0,0,PIN,0.843286,0,3,0.145551,7,5,1,10,3,0.272727,9949.742000,4274.848000,1.429500,10,1,0.097701,0.156714,0.142471,0
6992,6993,1924,22481,690,3,11,10760.85,CARD,0,Electronics,0.494317,0.835963,3,4401.59,1,0,0,NONE,0.585904,2,2,0.142240,15,0,0,14,0,0.000000,9310.800714,1450.049286,1.155614,14,1,0.085227,0.414096,0.108387,0
47909,47910,3183,22324,639,1,18,6677.40,CARD,0,Travel,0.267103,0.987481,5,8187.28,9,0,0,NONE,0.785155,1,0,0.061108,17,3,0,15,1,0.062500,8774.267333,-2096.867333,0.760934,15,1,0.077922,0.214845,0.085096,0
44385,44386,1034,24144,654,3,22,12069.48,UPI,0,Travel,0.348430,0.417403,10,9179.73,6,0,0,PIN,0.994131,1,0,0.065038,16,4,0,11,0,0.000000,6955.816364,5113.663636,1.734914,11,1,0.065476,0.005869,0.020817,0
22149,22150,4826,22055,653,3,12,19026.73,NETBANKING,0,Utilities,0.036590,0.257342,11,13075.65,9,0,0,OTP,0.601518,0,3,0.010849,11,1,0,16,1,0.058824,9849.987500,9176.742500,1.931454,16,1,0.110390,0.398482,0.130460,0
39278,39279,4133,25454,536,3,8,12019.80,UPI,0,Utilities,0.971669,0.001246,7,11336.56,1,0,0,PIN,0.710507,0,4,0.060346,1,4,0,10,0,0.000000,12598.674000,-578.874000,0.953977,10,1,0.074534,0.289493,0.080259,0
40753,40754,4538,29909,734,1,6,868.49,CARD,0,Fashion,0.962787,0.174552,3,7360.45,8,0,0,OTP,0.738705,2,0,0.145613,13,5,1,13,0,0.000000,7789.594615,-6921.104615,0.111479,13,1,0.077381,0.261295,0.075473,0
42510,42511,1631,26157,736,1,29,11724.05,UPI,0,Utilities,0.074746,0.197170,3,13353.67,8,0,0,PIN,0.983337,1,3,0.087225,18,0,0,9,0,0.000000,9425.971111,2298.078889,1.243671,9,1,0.091503,0.016663,0.030783,0
32457,32458,1969,28033,780,1,29,4758.50,UPI,0,Utilities,0.418426,0.185127,10,9623.71,4,0,0,OTP,0.667206,2,3,0.070450,9,0,0,13,0,0.000000,10157.249231,-5398.749231,0.468437,13,1,0.073620,0.332794,0.088645,0
7425,7426,4968,26157,500,2,26,4603.83,UPI,0,Fashion,0.489602,0.612045,1,14461.62,4,0,0,PIN,0.443890,2,2,0.003095,14,0,0,14,1,0.066667,11636.305000,-7032.475000,0.395610,14,1,0.061224,0.556110,0.149589,0


In [144]:
df.isnull().sum()

transaction_id                    0
customer_id                       0
device_id                         0
merchant_id                       0
transaction_month                 0
transaction_day                   0
amount                            0
payment_method                    0
is_international                  0
merchant_category                 0
ip_address_risk_score             0
device_trust_score                0
txn_count_last_24h                0
avg_amount_last_24h               0
merchant_diversity_last_7d        0
device_change_flag                0
location_change_flag              0
authentication_method             0
otp_success_rate_customer         0
past_fraud_count_customer         0
past_disputes_customer            0
merchant_historical_fraud_rate    0
hour_of_day                       0
day_of_week                       0
is_weekend                        0
cust_txn_count                    0
cust_fraud_count                  0
cust_fraud_rate             

In [145]:
df['cust_fraud_count'].value_counts()

cust_fraud_count
1    17383
0    13040
2    11676
3     5143
4     1974
5      608
6      140
7       36
Name: count, dtype: int64